## 重塑层次化索引
pandas的层次化也就是在一个轴上拥有多个索引级别，可以用函数使最外层的行索引作为DataFrame的行索引，内层的索引作为列索引，
比如对于一个股票数据可以设置股票名称为一级索引，年份日期为另一个索引。
为了更好进行建模和数据分析，我们常需要将数据排列保持一致，对数据进行重塑并基于分组的操作，层次化索引就起了很大的作用。  
 

In [4]:
import pandas as pd
import numpy as np
frame = pd.DataFrame(np.arange(12).reshape(4,3),index=[['a','a','b','b'],[1,2,1,2]],
            columns=[['ohio','ohio','color'],['green','red','green']]
            )
frame

ohio     color
    green red green
a 1     0   1     2
  2     3   4     5
b 1     6   7     8
  2     9  10    11

In [6]:
# 给层级行索引加名字
frame.index.names = ['key1','key2']

In [8]:
# 给层级列索引加名字
frame.columns.names = ['state','color']
frame

state      ohio     color
color     green red green
key1 key2                
a    1        0   1     2
     2        3   4     5
b    1        6   7     8
     2        9  10    11

In [10]:
frame['ohio']

color      green  red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

1、stack:将数据的列“旋转”为行  
2、unstack:将数据的行“旋转”为列

In [12]:
#用stack方法，将列转换为行，得到一个Series
result = frame.stack()
result

state            color  ohio
key1 key2 color             
a    1    green    2.0     0
          red      NaN     1
     2    green    5.0     3
          red      NaN     4
b    1    green    8.0     6
          red      NaN     7
     2    green   11.0     9
          red      NaN    10

In [14]:
#重排位一个DataFrame
result.unstack()


state     color      ohio    
color     green red green red
key1 key2                    
a    1      2.0 NaN     0   1
     2      5.0 NaN     3   4
b    1      8.0 NaN     6   7
     2     11.0 NaN     9  10

In [16]:
#传入分层级别的编号或名称即可对其他级别进行unstack操作
result.unstack(0)

state      color       ohio    
key1           a     b    a   b
key2 color                     
1    green   2.0   8.0    0   6
     red     NaN   NaN    1   7
2    green   5.0  11.0    3   9
     red     NaN   NaN    4  10

In [18]:
result.unstack('color')

state     color      ohio    
color     green red green red
key1 key2                    
a    1      2.0 NaN     0   1
     2      5.0 NaN     3   4
b    1      8.0 NaN     6   7
     2     11.0 NaN     9  10

In [23]:
#如果在分组中找不到级别值，则unstack操作会引入缺失数据：
from pandas import Series,DataFrame
s1=Series([0,1,2,3],index=['a','b','c','d'])
s2 = Series([4,5,6],index=['e','f','g'])
data2  = pd.concat([s1,s2],keys=['one','two'])
print(data2)
data2.unstack()

one  a    0
     b    1
     c    2
     d    3
two  e    4
     f    5
     g    6
dtype: int64


,a,b,c,d,e,f,g
one,0.0,1.0,2.0,3.0,NaN,NaN,NaN
two,NaN,NaN,NaN,NaN,4.0,5.0,6.0


In [25]:
#stack默认会滤除缺失数据
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  e    4.0
     f    5.0
     g    6.0
dtype: float64

In [27]:
#关闭滤除
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
     f    NaN
     g    NaN
two  a    NaN
     b    NaN
     c    NaN
     d    NaN
     e    4.0
     f    5.0
     g    6.0
dtype: float64

In [33]:
result.unstack('color').stack('state')

color            green   red
key1 key2 state             
a    1    color    2.0   NaN
          ohio     0.0   1.0
     2    color    5.0   NaN
          ohio     3.0   4.0
b    1    color    8.0   NaN
          ohio     6.0   7.0
     2    color   11.0   NaN
          ohio     9.0  10.0

## 将长格式旋转为宽格式
时间序列数据通常是以所谓的长格式或堆叠格式存储在数据库和csv中

In [83]:
docker = pd.read_csv('../data/platform/dcos_docker.csv')
docker

,itemid,name,bomc_id,timestamp,value,cmdb_id
0,999999996381369,container_thread_idle,ZJ-004-063,1586534406000,0.0,docker_002
1,999999996381255,container_session_used,ZJ-004-058,1586534417000,0.0,docker_005
2,999999996381449,container_cpu_used,ZJ-004-059,1586534402000,2.0,docker_004
3,999999996381323,container_thread_idle,ZJ-004-063,1586534423000,0.0,docker_008
4,999999996381393,container_session_used,ZJ-004-058,1586534428000,0.0,docker_003
...,...,...,...,...,...,...
25582,999999996381333,container_fgct,ZJ-004-104,1586555816000,0.0,docker_008
25583,999999996381367,container_thread_total,ZJ-004-064,1586555975000,0.0,docker_002
25584,999999996381367,container_thread_total,ZJ-004-064,1586555883000,0.0,docker_002
25585,999999996381367,container_thread_total,ZJ-004-064,1586555857000,0.0,docker_002


In [84]:
import datetime
docker['datetime'] = docker['timestamp'].apply( lambda x : datetime.datetime.fromtimestamp(x / 1000) )
docker['datetime']

0       2020-04-11 00:00:06
1       2020-04-11 00:00:17
2       2020-04-11 00:00:02
3       2020-04-11 00:00:23
4       2020-04-11 00:00:28
                ...        
25582   2020-04-11 05:56:56
25583   2020-04-11 05:59:35
25584   2020-04-11 05:58:03
25585   2020-04-11 05:57:37
25586   2020-04-11 05:56:37
Name: datetime, Length: 25587, dtype: datetime64[ns]

In [88]:
data = docker.loc[docker['cmdb_id']=='docker_002'][['datetime','name','value']]

In [89]:
data[:10]

,datetime,name,value
0,2020-04-11 00:00:06,container_thread_idle,0.0
10,2020-04-11 00:00:06,container_session_used,0.0
12,2020-04-11 00:00:06,container_fgct,0.0
15,2020-04-11 00:00:06,container_thread_running,0.0
21,2020-04-11 00:00:06,container_fgc,0.0
38,2020-04-11 00:00:06,container_thread_used_pct,0.0
39,2020-04-11 00:00:06,container_cpu_used,2.0
42,2020-04-11 00:00:06,container_thread_total,0.0
44,2020-04-11 00:00:06,container_mem_used,38.0
50,2020-04-11 00:05:06,container_thread_idle,0.0


In [90]:
#转为宽表
pivoted = data.pivot('datetime','name','value')
pivoted.head()

name,container_cpu_used,container_fgc,container_fgct,container_mem_used,container_session_used,container_thread_idle,container_thread_running,container_thread_total,container_thread_used_pct
datetime,,,,,,,,,
2020-04-11 00:00:06,2.0,0.0,0.0,38.0,0.0,0.0,0.0,0.0,0.0
2020-04-11 00:01:36,3.0,0.0,0.0,38.0,0.0,0.0,0.0,0.0,0.0
2020-04-11 00:02:06,1.0,0.0,0.0,38.0,0.0,0.0,0.0,0.0,0.0
2020-04-11 00:03:07,3.0,0.0,0.0,38.0,0.0,0.0,0.0,0.0,0.0
2020-04-11 00:04:08,3.0,0.0,0.0,38.0,0.0,0.0,0.0,0.0,0.0


In [92]:
data['value2'] = np.random.randn(len(data))
data[:10]

,datetime,name,value,value2
0,2020-04-11 00:00:06,container_thread_idle,0.0,0.673505
10,2020-04-11 00:00:06,container_session_used,0.0,-1.264237
12,2020-04-11 00:00:06,container_fgct,0.0,-0.888897
15,2020-04-11 00:00:06,container_thread_running,0.0,0.477402
21,2020-04-11 00:00:06,container_fgc,0.0,0.403999
38,2020-04-11 00:00:06,container_thread_used_pct,0.0,-0.022325
39,2020-04-11 00:00:06,container_cpu_used,2.0,0.154338
42,2020-04-11 00:00:06,container_thread_total,0.0,-0.072512
44,2020-04-11 00:00:06,container_mem_used,38.0,-0.731817
50,2020-04-11 00:05:06,container_thread_idle,0.0,1.188042


In [96]:
#如果忽略最后一个参数，得到的DataFrame就会带有层次化
pivoted = data.pivot('datetime','name')
pivoted[:5]

value                               \
name                container_cpu_used container_fgc container_fgct   
datetime                                                              
2020-04-11 00:00:06                2.0           0.0            0.0   
2020-04-11 00:01:36                3.0           0.0            0.0   
2020-04-11 00:02:06                1.0           0.0            0.0   
2020-04-11 00:03:07                3.0           0.0            0.0   
2020-04-11 00:04:08                3.0           0.0            0.0   

                                                               \
name                container_mem_used container_session_used   
datetime                                                        
2020-04-11 00:00:06               38.0                    0.0   
2020-04-11 00:01:36               38.0                    0.0   
2020-04-11 00:02:06               38.0                    0.0   
2020-04-11 00:03:07               38.0                    0.0   
2020-04-11 00:04:08               38.0                    0.0   

                                                                    \
name                container_thread_idle container_thread_running   
datetime                                                             
2020-04-11 00:00:06                   0.0                      0.0   
2020-04-11 00:01:36                   0.0                      0.0   
2020-04-11 00:02:06                   0.0                      0.0   
2020-04-11 00:03:07                   0.0                      0.0   
2020-04-11 00:04:08                   0.0                      0.0   

                                                                      \
name                container_thread_total container_thread_used_pct   
datetime                                                               
2020-04-11 00:00:06                    0.0                       0.0   
2020-04-11 00:01:36                    0.0                       0.0   
2020-04-11 00:02:06                    0.0                       0.0   
2020-04-11 00:03:07                    0.0                       0.0   
2020-04-11 00:04:08                    0.0                       0.0   

                                value2                               \
name                container_cpu_used container_fgc container_fgct   
datetime                                                              
2020-04-11 00:00:06           0.154338      0.403999      -0.888897   
2020-04-11 00:01:36          -0.549311      0.445057      -0.375740   
2020-04-11 00:02:06           0.177347      0.632433       0.566770   
2020-04-11 00:03:07          -0.341569     -0.809795      -0.190806   
2020-04-11 00:04:08          -0.681919      1.398485      -0.115224   

                                                               \
name                container_mem_used container_session_used   
datetime                                                        
2020-04-11 00:00:06          -0.731817              -1.264237   
2020-04-11 00:01:36          -0.458679               0.246456   
2020-04-11 00:02:06           0.129580              -1.173409   
2020-04-11 00:03:07           1.532339               1.848478   
2020-04-11 00:04:08           2.062885               1.238190   

                                                                    \
name                container_thread_idle container_thread_running   
datetime                                                             
2020-04-11 00:00:06              0.673505                 0.477402   
2020-04-11 00:01:36             -0.325963                -1.974367   
2020-04-11 00:02:06             -0.129180                -0.508956   
2020-04-11 00:03:07              0.259612                 1.331930   
2020-04-11 00:04:08              1.152982                -0.660629   

                                                                      
name                container_thread_total container_thread_used_pct  
datetime    

In [98]:
pivoted['value'][:5]

name,container_cpu_used,container_fgc,container_fgct,container_mem_used,container_session_used,container_thread_idle,container_thread_running,container_thread_total,container_thread_used_pct
datetime,,,,,,,,,
2020-04-11 00:00:06,2.0,0.0,0.0,38.0,0.0,0.0,0.0,0.0,0.0
2020-04-11 00:01:36,3.0,0.0,0.0,38.0,0.0,0.0,0.0,0.0,0.0
2020-04-11 00:02:06,1.0,0.0,0.0,38.0,0.0,0.0,0.0,0.0,0.0
2020-04-11 00:03:07,3.0,0.0,0.0,38.0,0.0,0.0,0.0,0.0,0.0
2020-04-11 00:04:08,3.0,0.0,0.0,38.0,0.0,0.0,0.0,0.0,0.0


In [100]:
#pivot 只是一个快捷方式
#用set_index
#再用unstack重塑

unstacked = data.set_index(['datetime','name']).unstack('name')

In [101]:
unstacked[:5]

value                               \
name                container_cpu_used container_fgc container_fgct   
datetime                                                              
2020-04-11 00:00:06                2.0           0.0            0.0   
2020-04-11 00:01:36                3.0           0.0            0.0   
2020-04-11 00:02:06                1.0           0.0            0.0   
2020-04-11 00:03:07                3.0           0.0            0.0   
2020-04-11 00:04:08                3.0           0.0            0.0   

                                                               \
name                container_mem_used container_session_used   
datetime                                                        
2020-04-11 00:00:06               38.0                    0.0   
2020-04-11 00:01:36               38.0                    0.0   
2020-04-11 00:02:06               38.0                    0.0   
2020-04-11 00:03:07               38.0                    0.0   
2020-04-11 00:04:08               38.0                    0.0   

                                                                    \
name                container_thread_idle container_thread_running   
datetime                                                             
2020-04-11 00:00:06                   0.0                      0.0   
2020-04-11 00:01:36                   0.0                      0.0   
2020-04-11 00:02:06                   0.0                      0.0   
2020-04-11 00:03:07                   0.0                      0.0   
2020-04-11 00:04:08                   0.0                      0.0   

                                                                      \
name                container_thread_total container_thread_used_pct   
datetime                                                               
2020-04-11 00:00:06                    0.0                       0.0   
2020-04-11 00:01:36                    0.0                       0.0   
2020-04-11 00:02:06                    0.0                       0.0   
2020-04-11 00:03:07                    0.0                       0.0   
2020-04-11 00:04:08                    0.0                       0.0   

                                value2                               \
name                container_cpu_used container_fgc container_fgct   
datetime                                                              
2020-04-11 00:00:06           0.154338      0.403999      -0.888897   
2020-04-11 00:01:36          -0.549311      0.445057      -0.375740   
2020-04-11 00:02:06           0.177347      0.632433       0.566770   
2020-04-11 00:03:07          -0.341569     -0.809795      -0.190806   
2020-04-11 00:04:08          -0.681919      1.398485      -0.115224   

                                                               \
name                container_mem_used container_session_used   
datetime                                                        
2020-04-11 00:00:06          -0.731817              -1.264237   
2020-04-11 00:01:36          -0.458679               0.246456   
2020-04-11 00:02:06           0.129580              -1.173409   
2020-04-11 00:03:07           1.532339               1.848478   
2020-04-11 00:04:08           2.062885               1.238190   

                                                                    \
name                container_thread_idle container_thread_running   
datetime                                                             
2020-04-11 00:00:06              0.673505                 0.477402   
2020-04-11 00:01:36             -0.325963                -1.974367   
2020-04-11 00:02:06             -0.129180                -0.508956   
2020-04-11 00:03:07              0.259612                 1.331930   
2020-04-11 00:04:08              1.152982                -0.660629   

                                                                      
name                container_thread_total container_thread_used_pct  
datetime    